In [ ]:
import pandas as pd

# Load the CSV files into DataFrames
schools_df = pd.read_csv('/mnt/data/schools_complete.csv')
students_df = pd.read_csv('/mnt/data/students_complete.csv')

# District Summary
total_schools = schools_df['school_name'].nunique()
total_students = students_df['Student ID'].count()
total_budget = schools_df['budget'].sum()
avg_math_score = students_df['math_score'].mean()
avg_reading_score = students_df['reading_score'].mean()
passing_math = students_df[students_df['math_score'] >= 70].count()['student_name']
percent_passing_math = (passing_math / total_students) * 100
passing_reading = students_df[students_df['reading_score'] >= 70].count()['student_name']
percent_passing_reading = (passing_reading / total_students) * 100
overall_passing = students_df[(students_df['math_score'] >= 70) & (students_df['reading_score'] >= 70)].count()['student_name']
percent_overall_passing = (overall_passing / total_students) * 100

district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [percent_passing_math],
    "% Passing Reading": [percent_passing_reading],
    "% Overall Passing": [percent_overall_passing]
})

# School Summary
school_groups = students_df.groupby('school_name')
school_types = schools_df.set_index('school_name')['type']
total_students_per_school = school_groups.size()
total_budget_per_school = schools_df.set_index('school_name')['budget']
per_student_budget = total_budget_per_school / total_students_per_school
avg_math_score_per_school = school_groups['math_score'].mean()
avg_reading_score_per_school = school_groups['reading_score'].mean()
percent_passing_math_per_school = school_groups.apply(lambda x: (x['math_score'] >= 70).sum() / x['Student ID'].count() * 100)
percent_passing_reading_per_school = school_groups.apply(lambda x: (x['reading_score'] >= 70).sum() / x['Student ID'].count() * 100)
percent_overall_passing_per_school = school_groups.apply(lambda x: ((x['math_score'] >= 70) & (x['reading_score'] >= 70)).sum() / x['Student ID'].count() * 100)

school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": total_students_per_school,
    "Total School Budget": total_budget_per_school,
    "Per Student Budget": per_student_budget,
    "Average Math Score": avg_math_score_per_school,
    "Average Reading Score": avg_reading_score_per_school,
    "% Passing Math": percent_passing_math_per_school,
    "% Passing Reading": percent_passing_reading_per_school,
    "% Overall Passing": percent_overall_passing_per_school
})

# Highest-Performing Schools (by % Overall Passing)
top_schools = school_summary.sort_values("% Overall Passing", ascending=False).head(5)

# Lowest-Performing Schools (by % Overall Passing)
bottom_schools = school_summary.sort_values("% Overall Passing").head(5)

# Math Scores by Grade
math_scores_by_grade = students_df.groupby(['school_name', 'grade'])['math_score'].mean().unstack()

# Reading Scores by Grade
reading_scores_by_grade = students_df.groupby(['school_name', 'grade'])['reading_score'].mean().unstack()

# Scores by School Spending
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], bins=spending_bins, labels=spending_labels)
spending_math_scores = school_summary.groupby("Spending Ranges (Per Student)")["Average Math Score"].mean()
spending_reading_scores = school_summary.groupby("Spending Ranges (Per Student)")["Average Reading Score"].mean()
spending_passing_math = school_summary.groupby("Spending Ranges (Per Student)")["% Passing Math"].mean()
spending_passing_reading = school_summary.groupby("Spending Ranges (Per Student)")["% Passing Reading"].mean()
overall_passing_spending = school_summary.groupby("Spending Ranges (Per Student)")["% Overall Passing"].mean()

spending_summary = pd.DataFrame({
    "Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending
})

# Scores by School Size
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_summary["School Size"] = pd.cut(school_summary["Total Students"], bins=size_bins, labels=size_labels)
size_math_scores = school_summary.groupby("School Size")["Average Math Score"].mean()
size_reading_scores = school_summary.groupby("School Size")["Average Reading Score"].mean()
size_passing_math = school_summary.groupby("School Size")["% Passing Math"].mean()
size_passing_reading = school_summary.groupby("School Size")["% Passing Reading"].mean()
overall_passing_size = school_summary.groupby("School Size")["% Overall Passing"].mean()

size_summary = pd.DataFrame({
    "Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing": overall_passing_size
})

# Scores by School Type
type_math_scores = school_summary.groupby("School Type")["Average Math Score"].mean()
type_reading_scores = school_summary.groupby("School Type")["Average Reading Score"].mean()
type_passing_math = school_summary.groupby("School Type")["% Passing Math"].mean()
type_passing_reading = school_summary.groupby("School Type")["% Passing Reading"].mean()
overall_passing_type = school_summary.groupby("School Type")["% Overall Passing"].mean()

type_summary = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": overall_passing_type
})

# Display dataframes
import ace_tools as tools
tools.display_dataframe_to_user(name="District Summary", dataframe=district_summary)
tools.display_dataframe_to_user(name="School Summary", dataframe=school_summary)
tools.display_dataframe_to_user(name="Top Schools", dataframe=top_schools)
tools.display_dataframe_to_user(name="Bottom Schools", dataframe=bottom_schools)
tools.display_dataframe_to_user(name="Math Scores by Grade", dataframe=math_scores_by_grade)
tools.display_dataframe_to_user(name="Reading Scores by Grade", dataframe=reading_scores_by_grade)
tools.display_dataframe_to_user(name="Spending Summary", dataframe=spending_summary)
tools.display_dataframe_to_user(name="Size Summary", dataframe=size_summary)
tools.display_dataframe_to_user(name="Type Summary", dataframe=type_summary)
